# KEGG Analysis

# Phase 2 vs. Phase 2

## 0. load libraries

In [2]:
library(tidyverse)
library(ggupset) 
library(enrichplot)
library(clusterProfiler) # for GSEA() and gseKEGG()

## 1. read CSVs

In [3]:
# get list of files
files <- list.files(
    path = '/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes',
    pattern = '\\.csv$',
    full.names = TRUE
    )

head(files)

[1] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bb_cc.csv"
[2] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bc_bb.csv"
[3] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bc_cc.csv"
[4] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bc_hc.csv"
[5] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bc_wc.csv"
[6] "/project/pi_sarah_gignouxwolfsohn_uml_edu/julia/CE_2024/CE24_RNA-seq/analysis/diff_expression/phase2_v_phase2/deseq_res_files/all_genes/bh_ch.csv"

`gseKEGG` takes in the whole list but uses LFC values (and wants them ordered, similar to GSEA - from what I've read, this is "better" than `enrichKEGG` bc of adding LFC, which allows your results to be more informative

In [4]:
names(files) <- gsub("^DEG_", "", tools::file_path_sans_ext(basename(files)))

df_list <- lapply(files, function(f) {
  df <- read.csv(f)

  # remove LOC from gene IDs
  df$Gene <- sub("^LOC", "", df$Gene)

  df
})

names(df_list) <- names(files)

all_df <- lapply(df_list, function(df) {
  geneList <- as.numeric(df$log2FoldChange)
  names(geneList) <- df$Gene

  # remove NA values
  geneList <- geneList[!is.na(geneList)]

  # sort decreasing
  sort(geneList, decreasing = TRUE)
})

names(all_df)
head(all_df$wc_cc)

[1] "bb_cc" "bc_bb" "bc_cc" "bc_hc" "bc_wc" "bh_ch" "bh_hh" "bw_cw" "bw_ww"
[10] "cb_bb" "cb_bc" "cb_cc" "cb_ch" "cb_cw" "ch_cc" "ch_hc" "cw_cc" "cw_ch"
[19] "cw_wc" "hb_bb" "hb_bh" "hb_cb" "hc_cc" "hc_hh" "hh_cc" "hh_ch" "wb_bb"
[28] "wb_bw" "wb_cb" "wc_cc" "wc_hc" "wc_ww" "wh_hw" "ww_cc" "ww_cw"

111132404 111105680 111112633 111103543 111113858 111109183 
 45.09222  43.67937  42.75454  40.28405  38.87249  34.30553

## 2. run `gseKEGG`

there are only two comparisons with enriched pathways - **CC vs. HC** and **CC vs. WW**

### 1. effect of single exposure after period of recovery
- CC vs. WC
- CC vs. HC
- CC vs. BC

In [44]:
# CC vs. WC
k.cc.wc <- gseKEGG(geneList = all_df$wc_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.wc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.54% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [45]:
# CC vs. HC
k.cc.hc <- gseKEGG(geneList = all_df$hc_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.hc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.57% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”


,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
cvn04144,cvn04144,Endocytosis - Crassostrea virginica (eastern oyster),214,0.9957203,1.341435,0.000172953,0.02421342,0.02421342,56,"tags=1%, list=0%, signal=1%",111110117/111099630


endocytosis - cell membrane engulfs external substances, carries material inside the cell

enriched pathway in HC (I think that's how you interpret...)

enriched GO terms for this pathway are involved in cell migration/organization

In [47]:
# CC vs. BC
k.cc.bc <- gseKEGG(geneList = all_df$bc_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.bc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.55% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

### 2. effect of single late exposure 
- CC vs. CW
- CC vs. CH
- CC vs. CB

(no enriched pathways from any of these comparisons)

In [48]:
# CC vs. CW
k.cc.cw <- gseKEGG(geneList = all_df$cw_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.cw)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.77% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [49]:
# CC vs. CH
k.cc.ch <- gseKEGG(geneList = all_df$ch_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.ch)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.6% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [50]:
# CC vs. CB
k.cc.cb <- gseKEGG(geneList = all_df$cb_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.cb)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.75% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

### 3. Effect of timing of initial stress exposure
- CW vs. WC
- CH vs. HC
- CB vs. BC

(no enriched pathways from any of these comparisons)

In [51]:
# CW vs. WC
k.cw.wc <- gseKEGG(geneList = all_df$cw_wc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cw.wc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.62% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [52]:
# CH vs. HC
k.ch.hc <- gseKEGG(geneList = all_df$ch_hc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.ch.hc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.41% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [53]:
# CB vs. BC
k.cb.bc <- gseKEGG(geneList = all_df$cb_bc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cb.bc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.51% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

### 4. No stress vs. most stress
- CC vs. WW
- CC vs. HH
- CC vs. BB

In [5]:
# CC vs. WW
k.cc.ww <- gseKEGG(geneList = all_df$ww_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.ww)

Reading KEGG annotation online: "https://rest.kegg.jp/link/cvn/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/cvn"...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.76% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”


,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue,rank,leading_edge,core_enrichment
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
cvn00830,cvn00830,Retinol metabolism - Crassostrea virginica (eastern oyster),51,-0.9994847,-1.356963,8.69882e-05,0.01217835,0.01217835,9,"tags=4%, list=0%, signal=4%",111102792/111100601
cvn04144,cvn04144,Endocytosis - Crassostrea virginica (eastern oyster),214,0.9964677,1.260029,4.89459e-04,0.03426213,0.03426213,51,"tags=1%, list=0%, signal=1%",111110117/111099630


- endocytosis - cell engulfs substance and brings it inside
    - seen in CC vs. HC

In [6]:
# CC vs. HH
k.cc.hh <- gseKEGG(geneList = all_df$hh_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.hh)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.71% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [7]:
# CC vs. BB
k.cc.bb <- gseKEGG(geneList = all_df$bb_cc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cc.bb)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.53% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

### 5. Effect of initial exposure on response in later exposure

#### 5A. matched priming
- WW vs. CW
- HH vs. CH
- BB vs. CB

(no enriched pathways from any of these comparisons)

In [8]:
# CW vs. WW
k.cw.ww <- gseKEGG(geneList = all_df$ww_cw,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cw.ww)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.81% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [9]:
# CH vs. HH
k.ch.hh <- gseKEGG(geneList = all_df$hh_ch,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.ch.hh)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.55% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [11]:
# CB vs. BB
k.cb.bb <- gseKEGG(geneList = all_df$cb_bb,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cb.bb)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.5% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

#### 5B. semi-matched priming: in context of later *single* stress
- CH vs. BH
- CW vs. BW

(no enriched pathways in these comparisons)

In [13]:
# CH vs. BH
k.ch.bh <- gseKEGG(geneList = all_df$bh_ch,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.ch.bh)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.39% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [14]:
# CW vs. BW
k.cw.bw <- gseKEGG(geneList = all_df$bw_cw,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cw.bw)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.84% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

#### 5C. semi-matched priming: in context of later *multiple* stress
- CB vs. WB
- CB vs. HB

(no enriched pathways in these comparisons)

In [16]:
# CB vs. WB
k.cb.wb <- gseKEGG(geneList = all_df$wb_cb,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cb.wb)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.83% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [17]:
# CB vs. HB
k.cb.hb <- gseKEGG(geneList = all_df$hb_cb,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cb.hb)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.78% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

### 6. Effect of initial exposure on subsequent response to stress
- WW vs. WC
- HH vs. HC
- BB vs. BC

(no enriched pathways in these comparisons)

In [20]:
# WW vs. WC
k.wc.ww <- gseKEGG(geneList = all_df$wc_ww,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.wc.ww)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.6% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [21]:
# HH vs. HC
k.hc.hh <- gseKEGG(geneList = all_df$hc_hh,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.hc.hh)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.5% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [22]:
# BB vs. BC
k.bc.bb <- gseKEGG(geneList = all_df$bc_bb,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.bc.bb)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.51% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

### 7. Effect of initial single/multiple stress on response in ...

#### 7A. ... later *single* stress
- BH vs. HH
- BW vs. WW

(no enriched pathways in these comparisons)

In [23]:
# BH vs. HH
k.bh.hh <- gseKEGG(geneList = all_df$bh_hh,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.bh.hh)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.5% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [24]:
# BW vs. WW
k.bw.ww <- gseKEGG(geneList = all_df$bw_ww,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.bw.ww)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.75% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

#### 7B. ... later *multiple* stressors
- HB vs. BB
- WB vs. BB

In [25]:
# HB vs. BB
k.hb.bb <- gseKEGG(geneList = all_df$hb_bb,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.hb.bb)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.51% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [26]:
# WB vs. BB
k.wb.bb <- gseKEGG(geneList = all_df$wb_bb,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.wb.bb)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.54% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

### 8. Shared/unique responses among stressors with ...

#### 8A. different *early* stressors
- WC vs. HC
- BC vs. HC
- BC vs. WC

(no enriched pathways in these comparisons)

In [27]:
# WC vs. HC
k.wc.hc <- gseKEGG(geneList = all_df$wc_hc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.wc.hc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.41% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [28]:
# BC vs. HC
k.bc.hc <- gseKEGG(geneList = all_df$bc_hc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.bc.hc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.38% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [29]:
# BC vs. WC
k.bc.wc <- gseKEGG(geneList = all_df$bc_wc,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.bc.wc)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.38% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

#### 8B. different *later* stress
- CW vs. CH
- CB vs. CH
- CW vs. CB

(no enriched pathways in these comparisons)

In [30]:
# CW vs. CH
k.cw.ch <- gseKEGG(geneList = all_df$cw_ch,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cw.ch)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.64% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [31]:
# CB vs. CH
k.cb.ch <- gseKEGG(geneList = all_df$cb_ch,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cb.ch)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.63% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [32]:
# CB vs. CW
k.cb.cw <- gseKEGG(geneList = all_df$cb_cw,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.cb.cw)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.81% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

### 9. Effect of order of stresses on GE
- WH vs. HW
- WB vs. BW
- HB vs. BH

(no enriched pathways in these comparisons)

In [33]:
# WH vs. HW
k.wh.hw <- gseKEGG(geneList = all_df$wh_hw,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.wh.hw)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.51% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [34]:
# WB vs. BW
k.wb.bw <- gseKEGG(geneList = all_df$wb_bw,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.wb.bw)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.79% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways

In [35]:
# HB vs. BH
k.hb.bh <- gseKEGG(geneList = all_df$hb_bh,
               organism     = 'cvn',
              # minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
as.data.frame(k.hb.bh)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.55% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
no term enriched under specific pvalueCutoff...



ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalue
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


no enriched pathways